In [ ]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

try:
    conn = mysql.connector.connect(
        host='localhost',       
        user='root',            
        password='',            
        database='loja_vendas',
        port= 3306  
    )
    print("Conexão estabelecida com sucesso!")
except mysql.connector.Error as err:
    print(f"Erro ao conectar: {err}")
    exit()

Conexão estabelecida com sucesso!


In [9]:
# Função para executar consultas e retornar DataFrames
def consultar_sql(query):
    return pd.read_sql(query, conn)

# Consulta 1: Total de vendas por produto
query_vendas_produto = '''
SELECT p.nome_produto, SUM(v.quantidade) AS total_vendido
FROM produtos p
JOIN vendas v ON p.id_produto = v.id_produto
GROUP BY p.nome_produto
ORDER BY total_vendido DESC
'''
df_vendas_produto = consultar_sql(query_vendas_produto)

# Consulta 2: Gastos por cliente
query_gastos_cliente = '''
SELECT c.nome, SUM(v.quantidade * p.preco) AS total_gasto
FROM clientes c
JOIN vendas v ON c.id_cliente = v.id_cliente
JOIN produtos p ON v.id_produto = p.id_produto
GROUP BY c.nome
ORDER BY total_gasto DESC
'''
df_gastos_cliente = consultar_sql(query_gastos_cliente)

# Consulta 3: Receita por dia
query_vendas_dia = '''
SELECT v.data_venda, SUM(v.quantidade * p.preco) AS receita_diaria
FROM vendas v
JOIN produtos p ON v.id_produto = p.id_produto
GROUP BY v.data_venda
ORDER BY v.data_venda
'''
df_vendas_dia = consultar_sql(query_vendas_dia)




C:\Users\Jheferson Warley\AppData\Local\Temp\ipykernel_10716\2488523396.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


In [ ]:
# Consultas para pegar todas as informações das tabelas
query_clientes = "SELECT * FROM clientes"
query_produtos = "SELECT * FROM produtos"
query_vendas = "SELECT * FROM vendas"

# Executar as consultas e armazenar os resultados em DataFrames
df_clientes = consultar_sql(query_clientes)
df_produtos = consultar_sql(query_produtos)
df_vendas = consultar_sql(query_vendas)


conn.close()

# Criar um arquivo Excel com múltiplas abas
with pd.ExcelWriter('relatorio_loja_vendas.xlsx') as writer:
    df_clientes.to_excel(writer, sheet_name='Clientes', index=False)
    df_produtos.to_excel(writer, sheet_name='Produtos', index=False)
    df_vendas.to_excel(writer, sheet_name='Vendas', index=False)

print("Dados exportados para 'relatorio_loja_vendas.xlsx' com sucesso!")

Dados exportados para 'relatorio_loja_vendas.xlsx' com sucesso!


C:\Users\Jheferson Warley\AppData\Local\Temp\ipykernel_10716\2488523396.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


In [11]:
# Tratamento de dados
df_vendas_dia['data_venda'] = pd.to_datetime(df_vendas_dia['data_venda'])
display(df_vendas_dia)

,data_venda,receita_diaria
0,2025-03-01,119.6
1,2025-03-02,399.6
2,2025-03-03,239.6
3,2025-03-04,239.7
4,2025-03-05,59.7
5,2025-03-06,59.8
6,2025-03-07,499.5
7,2025-03-08,299.5
8,2025-03-09,159.8
9,2025-03-10,59.7


In [ ]:
# Gráfico 1: Total vendido por produto (barras)
fig1 = px.bar(df_vendas_produto, x='nome_produto', y='total_vendido', 
              title='Total Vendido por Produto', 
              labels={'nome_produto': 'Produto', 'total_vendido': 'Quantidade Vendida'},
              color='total_vendido',  
              color_continuous_scale=list((px.colors.sequential.Blues)))
fig1.update_traces(hovertemplate='Produto: %{x}<br>Quantidade: %{y}')
fig1.update_layout(xaxis_tickangle=-45, height=600)
fig1.show()

# Gráfico 2: Top 10 clientes por gasto (barras)
fig2 = px.bar(df_gastos_cliente.head(10), x='nome', y='total_gasto', 
              title='Top 10 Clientes por Gasto', 
              labels={'nome': 'Cliente', 'total_gasto': 'Total Gasto (R$)'},
              color='total_gasto', 
              color_continuous_scale=list((px.colors.sequential.Greens)))
fig2.update_traces(hovertemplate='Cliente: %{x}<br>Gasto: R$%{y:.2f}')
fig2.update_layout(xaxis_tickangle=-45, height=600)
fig2.show()

# Gráfico 3: Receita diária (linha)
fig3 = px.line(df_vendas_dia, x='data_venda', y='receita_diaria', 
               title='Receita Diária', 
               labels={'data_venda': 'Data', 'receita_diaria': 'Receita (R$)'},
               markers=True, 
               color_discrete_sequence=['purple'])
fig3.update_traces(hovertemplate='Data: %{x}<br>Receita: R$%{y:.2f}')
fig3.update_layout(xaxis_tickformat='%Y-%m-%d', height=600)
fig3.show()


Gráficos interativos gerados com sucesso!
